In [1]:
'''
参考代码：https://blog.csdn.net/weixin_42608414/article/details/88391760
'''
import re
#定义删除除字母,数字，汉字以外的所有符号的函数
def remove_punctuation(line):
    line = str(line)
    if line.strip()=='':
        return ''
    rule = re.compile(u"[^a-zA-Z0-9\u4E00-\u9FA5]")
    line = rule.sub('',line)
    return line

#停用词列表
def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]
    return stopwords

#加载停用词
stopwords = stopwordslist("./stopwords.txt")

In [2]:
import pandas as pd
# df = pd.read_csv('./data_lite_all.csv')
df = pd.read_csv('../data_cleaned_1.csv')
df=df[['text','label']]
print("数据总量: %d ." % len(df))
df.sample(10)

数据总量: 2000 .


,text,label
1986,亲爱的老师，解释一下为什么是这样子的谢谢,3
1837,这题怎么做呀，谢谢啦,3
634,表示这道题不理解,1
927,刚刚在网课上看到的,1
693,不过你是怎么倒过来了一下,1
571,直接洛必达一下就解决了啊,1
1880,向老师求助请老师帮忙解答一下这三道题,3
1631,咋做呢，老师,3
984,因为是任意给定的E,1
1289,老师，这些是为什么啊求解，,2


In [3]:
import jieba as jb
#删除除字母,数字，汉字以外的所有符号
df['clean_text'] = df['text'].apply(remove_punctuation)

#分词，并过滤停用词
# df['cut_review'] = df['clean_review'].apply(lambda x: " ".join([w for w in list(jb.cut(x)) if w not in stopwords]))
df['cut_text'] = df['clean_text'].apply(lambda x: [w for w in list(jb.cut(x)) if w not in stopwords])
df.head()

Building prefix dict from the default dictionary ...
Loading model from cache C:\WINDOWS\TEMP\jieba.cache
Loading model cost 0.491 seconds.
Prefix dict has been built successfully.


,text,label,clean_text,cut_text
0,为什么无界函数不一定为无穷大,0,为什么无界函数不一定为无穷大,"[为什么, 无, 界, 函数, 不, 一定, 为, 无穷大]"
1,判断，处有无定义，若，处有定义，但是，不存在，即，不存在或，不存在，若，处有定义存在，但是,0,判断处有无定义若处有定义但是不存在即不存在或不存在若处有定义存在但是,"[判断, 处, 有无, 定义, 若处, 有, 定义, 但是, 不, 存在, 即, 不, 存在..."
2,谢谢,0,谢谢,[谢谢]
3,先开上面三次方再求导,0,先开上面三次方再求导,"[先开, 上面, 三次方, 再, 求导]"
4,求解答这里是讲收敛数列的有界性，为什么举例是这个例子这个数列并不收敛呀这个数列不是发散的吗,0,求解答这里是讲收敛数列的有界性为什么举例是这个例子这个数列并不收敛呀这个数列不是发散的吗,"[求, 解答, 这里, 是, 讲, 收敛, 数列, 的, 有界性, 为什么, 举例, 是, ..."


In [4]:
from gensim.models.doc2vec import TaggedDocument
#创建标签化文档
train_tagged = df.apply(
    lambda r: TaggedDocument(words=r['cut_text'], tags=[r['label']]), axis=1)
train_tagged.sample(10)

285     ([老师, 这个, 条件, 是, 么, 意思, 是, 确保, gx, 的, 极限, 为, u...
1076                                ([这题, 怎么, 做, 啊], [2])
164     ([如果, 我, 是, 你, 对于, 别人, 的, 这个, 问题, 我会, 回答, 分, X...
56                                            ([谢谢], [0])
1461                           ([为什么, 不能, 这么, 做, 呢], [2])
1084                               ([第十一, 题, 怎么, 写], [2])
156                        ([这道题, 有无, 更, 机智, 的, 做法], [0])
1433                         ([咋, 做, 还是, 没看, 懂, 求教], [2])
1230                                 ([那, 怎么, 求, 呢], [2])
945     ([用, 放缩, 当, x, 趋于, 正, 无穷, 时, sinxxxx, 埃普, 西隆],...
dtype: object

In [36]:
import multiprocessing
from gensim.models import Doc2Vec
from tqdm import tqdm

cores = multiprocessing.cpu_count()
# model_dbow = Doc2Vec(dm=1, vector_size=300, negative=5, hs=0, min_count=2, sample = 1e-3, workers=cores, window=1)  # 0 &.7
model_dbow = Doc2Vec(dm=1, vector_size=512, negative=5, hs=0, min_count=2, sample = 1e-3, workers=cores, window=1)  # 0 &.7
model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

100%|██████████| 2000/2000 [00:00<00:00, 1999668.18it/s]


In [37]:
from sklearn import utils
# for epoch in tqdm(range(10)):
#     model_dbow.train(train_tagged.values, total_examples=len(train_tagged.values), epochs=30)
for epoch in tqdm(range(1000)):
    # model_dbow.train(utils.shuffle([x for x in train_tagged.values]), total_examples=len(train_tagged.values), epochs=10)
    model_dbow.train(utils.shuffle([x for x in train_tagged.values]), total_examples=len(train_tagged.values), epochs=10)
    # model_dbow.train(train_tagged.values, total_examples=len(train_tagged.values), epochs=7)
    model_dbow.alpha -= 0.002
    # model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 1000/1000 [10:10<00:00,  1.64it/s]


In [38]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words)) for doc in sents])
    return targets, regressors

y_train, X_train = vec_for_learning(model_dbow, train_tagged)

In [39]:
import numpy as np
X_train = list(X_train)
y_train = np.array(y_train)
X_train = X_train[:3664]
y_train = y_train[:3664]
len(y_train),len(X_train)

(2000, 2000)

In [48]:
import numpy as np

total = np.concatenate((X_train, y_train.reshape(len(y_train), 1)), axis=1)
tp = pd.DataFrame(total)
tp.to_csv("../d2v_cleaned_2000.csv", header=None, index=None)
# tp.to_csv("d2v.csv", header=None, index=None)

In [40]:
# 后端网络LPA分类
import numpy as np
from sklearn.semi_supervised import LabelSpreading
from sklearn.metrics import classification_report, accuracy_score

data = np.concatenate((X_train, y_train.reshape(len(y_train), 1)), axis=1)
print(data.shape)
data = data[:2000]
X = data[:, : -1]  # 输入
y = data[:, -1]  # 输出
rng = np.random.RandomState(5)  # 随机数种子
indices = np.arange(len(data))
rng.shuffle(indices)  # 将索引打乱
# print(X[0])

n_total_samples = len(y)
# n_labeled_points = 2795  # 已标记/样本数
n_labeled_points = 1600  # 已标记样本数1260 1440

unlabeled_set = indices[n_labeled_points:]

y_train = np.copy(y)
y_train[unlabeled_set] = -1  # 未标记样本集y设为-1

num=100
accuracy = []
gammas = np.logspace(-2, 3, num=num)
# gammas = range(100)
score_in = 0
for i in range(num):
    lp_model = LabelSpreading(gamma=gammas[i], max_iter=30, kernel='rbf')
    lp_model.fit(X, y_train)
    predicted_labels = lp_model.transduction_[unlabeled_set]
    true_labels = y[unlabeled_set]
    score = accuracy_score(true_labels, predicted_labels)
    # if score > score_in:
    #     gamma_fin = gammas[i]
    accuracy.append(score)
    print(i,gammas[i],score)
    # print(classification_report(true_labels, predicted_labels, digits=4))

# lp_model = LabelSpreading(gamma=0.24, max_iter=200, kernel='rbf')# bert2000
lp_model = LabelSpreading(gamma=1.9, max_iter=200, kernel='rbf')# tf-idf2000
# lp_model = LabelSpreading(kernel='knn', n_neighbors=9, max_iter=100)
lp_model.fit(X, y_train)
predicted_labels = lp_model.transduction_[unlabeled_set]
true_labels = y[unlabeled_set]
print(classification_report(true_labels, predicted_labels, digits=4))



# print(
#     "Label Spreading model: %d labeled & %d unlabeled points (%d total)"
#     % (n_labeled_points, n_total_samples - n_labeled_points, n_total_samples)
# )
# print(classification_report(true_labels, predicted_labels))
# print(true_labels)
# print(predicted_labels)


(2000, 513)
0 0.01 0.3225
1 0.011233240329780276 0.3225
2 0.012618568830660204 0.335
3 0.014174741629268055 0.3425
4 0.015922827933410922 0.34
5 0.01788649529057435 0.345
6 0.02009233002565047 0.355
7 0.022570197196339202 0.3575
8 0.025353644939701114 0.3525
9 0.02848035868435802 0.345
10 0.03199267137797385 0.35
11 0.03593813663804628 0.35
12 0.040370172585965536 0.3525
13 0.04534878508128582 0.355
14 0.0509413801481638 0.345
15 0.05722367659350217 0.35
16 0.06428073117284319 0.3475
17 0.07220809018385464 0.3425
18 0.08111308307896872 0.345
19 0.09111627561154892 0.34
20 0.10235310218990264 0.3475
21 0.11497569953977356 0.3525
22 0.1291549665014884 0.3525
23 0.14508287784959395 0.3425
24 0.16297508346206444 0.34
25 0.18307382802953678 0.335
26 0.20565123083486514 0.3325
27 0.23101297000831592 0.3325
28 0.25950242113997357 0.335
29 0.2915053062825176 0.335
30 0.32745491628777285 0.3325
31 0.36783797718286326 0.3325
32 0.41320124001153363 0.33
33 0.464158883361278 0.3325
34 0.5214008287

In [29]:
data.shape
data[:10]

array([[-4.74266335e-02, -3.02468613e-03, -7.81949237e-02,
        -4.59347805e-03, -7.25578982e-03,  1.51773486e-02,
         1.53552219e-02,  6.67010918e-02, -3.50089818e-02,
         2.71189213e-02,  4.79480214e-02,  5.97529449e-02,
         3.76101807e-02, -3.28362845e-02,  6.18035607e-02,
        -4.01651636e-02,  3.46981920e-02,  4.31404868e-03,
        -1.98813081e-02, -2.92790588e-02, -4.01854627e-02,
         9.44314897e-03, -6.76565543e-02, -5.07358089e-02,
        -3.45638879e-02,  2.20429190e-02, -1.13179915e-01,
        -1.29152248e-02,  3.49431299e-03,  2.92934328e-02,
         7.60602504e-02,  4.07384746e-02, -7.12106675e-02,
         1.30992448e-02,  6.23914637e-02,  2.90053822e-02,
         3.26382183e-02, -5.06384969e-02,  3.88111956e-02,
        -3.36839370e-02, -3.57929058e-02,  3.27311195e-02,
        -6.46381602e-02, -8.98270682e-02, -7.02664070e-03,
         5.33068776e-02,  4.42453846e-02,  1.14518432e-02,
         6.29458576e-02, -3.77650708e-02,  0.00000000e+0